# EART 252 -- Homework 1

In [1]:
from importlib import reload

import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns

import unyt

In [2]:
sns.set_theme()
sns.set_style('whitegrid')
sns.set_context('notebook')

----

# Problem 1

In [3]:
# Net radiative heating
R_T = 98 * unyt.W / unyt.m**2

## In steady state, averaged over the Earth (with $E=P$), we derived:

## $L_V P = -R - SH$

## Assuming that $SH$ is small relative to $R$, we have:

## $P = -R / L_V$

In [4]:
# L_V
L_V = 2.45 * 10**6 * unyt.J / unyt.kg

## But we are told, the evaporative fraction is 0.8 so I take that to mean:

## $P = -\epsilon R / L_V$

## with $\epsilon = 0.8$

## Therefore, calculate $p$

In [5]:
# We ignore the sign
epsilon = 0.8
p = epsilon * R_T / L_V
p.in_cgs()

unyt_quantity(3.2e-06, 'g/(cm**2*s)')

## Now one cubic centimeter of water is 1 gram

## m/s

In [6]:
P = p * (1*unyt.cm**3/unyt.g)
P.to('m/s')

unyt_quantity(3.2e-08, 'm/s')

## mm/day

In [7]:
P.to('mm/day')

unyt_quantity(2.7648, 'mm/day')

## m/yr

In [8]:
P.to('m/yr')

unyt_quantity(1.0098432, 'm/yr')

----

# Problem 2 -- Greenhouse warming

## Warming

## Increase $R_T$ by 0.8 W m$^{-2}$

## And $\epsilon$ by 0.012

In [9]:
R_T_2 = R_T + 0.8 * unyt.W / unyt.m**2
epsilon_2 = epsilon + 0.012

## Recalculate

In [10]:
p_2 = epsilon_2 * R_T_2 / L_V

## Fractional change

In [11]:
(p_2-p)/p

unyt_quantity(0.02328571, '(dimensionless)')

## $\approx 2\%$

----

# Problem 3 -- Sahara subsiding

## In class, we derived a regional expression:

## $L_V P = -R - SH - \nabla \cdot F_{e,dry}$

## where the latter term has both horizontal and vertical components.

## Here we will only consider the horizontal.

## And, for the Sahara we will assume 0 precipitation i.e. $P=0$ and ignore SH.

## Therefore, 

## $R  = \int w \frac{\partial s}{\partial p} \frac{dp}{g}$

### with $s = c_pT + gz$

## Examining the Annual figure, I eyeball that the radiative cooling is $R \approx -20 \rm W/m^2$

In [12]:
# R
R = -20 * unyt.W/unyt.m**2

## We are told to ignore the temperature term in the static energy, i.e.

## $s \approx gz$

## At the Earth's surface, I recall that the density profile is nearly exponential.  And we saw the temperature profile is not steep (i.e. $T$ is approximately constant).

## This means, approximately, that:

## $z = \frac{R_V T}{g} \, \ln p$

## Therefore, we can write:

## $\frac{\partial s}{\partial p} = \frac{\partial s}{\partial z} \frac{\partial z}{\partial p}$

## Giving

## $\frac{\partial s}{\partial p} = \frac{R_V T}{p}$

## That leaves:

## $R  = \int w \frac{R_V T}{g} \frac{dp}{p}$

## Am a bit nervous on the units/constants above, but we'll see how this goes..

## Integrating assuming a constant vertical wind, 

## $R = \frac{w R_V T}{g} \ln \frac{p_1}{p_0}$

## with $p_1, p_0$ the pressure up high and at the Earth's surface.

## Time for numerics

In [13]:
# Values at the Earth's surface
g = 9.8 * unyt.m/unyt.s**2
R_V = 287 * unyt.J / unyt.K / unyt.kg# Specific
T = 288 * unyt.K # Probably a bit hotter in the Sahara..
#  Guess at p1/p0
p1_p0 = 1./2.
p0 = 1000 * unyt.hPa

## Here I weight the radiative cooling by $g^{-1}$ and not $p_s/g$.  Am sure that leads to trouble below but I can't get the units to make sense otherwise..

In [16]:
w = (R/g)  * g / R_V / T / np.log(p1_p0)  # I have weighted R by 1/g not ps/g
w.in_cgs()

unyt_quantity(3.49084166e-05, 'g/(cm**2*s)')

## To get the units right, need to multiply the number of particles in a cubic cm of air

In [17]:
air_density = 1.27 * unyt.kg / unyt.m**3

In [19]:
w_final = w / air_density
w_final.to('km/hr')

unyt_quantity(0.00098953, 'km/hr')

## That seems awfully slow, although the $w$ value in Problem 5 is only 10x faster..

## Still, I suspect my weighting $R$ by only $g^{-1}$ is flawed..

----

# Problem 4 -- Rain in the Sahara

## The net radiaton is given by:

## $R_T = \rm ASR - OLR$ 

## which we have eyeballed as a net loss of $-20 \rm W/m^2$

## Now, the absorbed solar radiation (ASR) is given by:

## ${\rm ASR} = \frac{(1-\alpha)S_\odot}{4}$

## with $\alpha$ the albedo and $S_\odot = 1386 \rm \, W/m^2$ is the solar constant

## And, the outgoing longwavelenth radiation (OLR) is given by:

## ${\rm OLR} = \sigma T_e^4$

In [20]:
S_o = 1386 * unyt.W / unyt.m**2

## Perturbing our equation,

## $\delta R_T = \rm \delta ASR - \delta OLR$

## Giving

## $\delta R_T = -\delta \alpha S_\odot/4 - 4 \sigma T_e^3 \delta T_E$

### Consider the 2nd term for a moment

### For $\delta T_E = 1$K, we have:

In [21]:
T_e = 300 * unyt.K
sigma = 5.6 * 10**(-8) * unyt.W / unyt.m**2 / unyt.K**4
BB = 4 * sigma * T_e**3 * (1*unyt.K)
BB

unyt_quantity(6.048, 'W/m**2')

### So, if the desert heats by only 1K, it is a not-so-important term.  3K, however...

## We'll ignore the 2nd term and thereby derive:

## $\delta \alpha = 4 R_T / S_\odot$

In [22]:
dalpha = 4 * (40*unyt.W/unyt.m**2) / S_o
dalpha

unyt_quantity(0.11544012, '(dimensionless)')

## Therefore, decreasing the albedo by an absolute amount of 0.11 would be sufficient.  But that's a big change!

## One could account for the surface temperature increase by the vertical winds of Problem 3.  Otherwise, I think you'd have to further adjust the albedo.

----

# Problem 5 -- 

## We wish to force a precipitation $P = 1 \, \rm m \, yr^{-1}$ by forcing a vertical transport of air with vertical velocity $w$.

## By conservation of water, we have:

## $(w \rho_a q)_{\rm 1 km} = P \rho_w$

## with $\rho_a, \rho_w$ the density of air and liquid water and $q$ is the specific humidity.

In [24]:
# Define
rho_a = 1.27 * unyt.kg / unyt.m**3
rho_w = 1 * unyt.g / unyt.cm**3
P = 1 * unyt.m / unyt.yr
q = 12*unyt.g / (12*unyt.g + 1*unyt.kg)

In [27]:
w_2 = P * rho_w / rho_a / q
w_2.to('km/hr')

unyt_quantity(0.0075752, 'km/hr')

## Taking our answer for Problem 3 as correct (I am doubtful), we derived $w=0.0009 \, \rm km/hr$ for $R = 20 \, \rm W \, m^{-2}$

## This scaling means:

In [29]:
R_needed = w_2 * (20 * unyt.W / unyt.m**2) / (0.0009 * unyt.km/unyt.hr)
R_needed.to('W/m**2')

unyt_quantity(168.33776833, 'W/m**2')

## That means a change of $\approx 188 \, \rm W \, m^{-2}$

In [30]:
dalpha_2 = 4 * (180*unyt.W/unyt.m**2) / S_o
dalpha_2

unyt_quantity(0.51948052, '(dimensionless)')

## Whoa!

## Looking at Fig 1, the Amazon peaks at $\approx 100 \, \rm W \, m^{-2}$ which is a bit below the $168 \, \rm W \, m^{-2}$ that I estimated.